In [38]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

remove_fight_island = True

In [39]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 13

5


In [40]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  'DecisionTreeClassifier(max_depth=5, max_leaf_nodes=5, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)',
  'GradientBoostingClassifier(max_

In [41]:
df = pd.read_csv("../data/ufc-master.csv")

In [42]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [43]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [44]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [45]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4243
4243
4243


In [46]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


93
4231


In [10]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Raquel Pennington,Marion Reneau,-160,140,62.500000,140.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,4.0,0.50,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0
201,Belal Muhammad,Lyman Good,-130,110,76.923077,110.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.25,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0
202,Jim Miller,Roosevelt Roberts,200,-240,200.000000,41.666667,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.00,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0
203,Clay Guida,Bobby Green,195,-235,195.000000,42.553191,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,0.25,0.66,0.0,2.0,0.0,0.0,0.0,0.0,1
204,Tecia Torres,Brianna Van Buren,175,-210,175.000000,47.619048,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,7.0,0.00,0.28,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.50,0.0,0.0,0.0,0.0,0.0,0.0,1
4439,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.00,0.0,1.0,0.0,1.0,0.0,1.0,0
4440,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
4441,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.00,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
56,Colby Covington,Tyron Woodley,-335,260,29.850746,260.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
57,Khamzat Chimaev,Gerald Meerschaert,-400,300,25.000000,300.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
58,Johnny Walker,Ryan Spann,-125,100,80.000000,100.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
59,Mackenzie Dern,Randa Markos,-177,140,56.497175,140.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
60,Kevin Holland,Darren Stewart,-265,205,37.735849,205.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Takashi Sato,Jason Witt,-278,215,35.971223,215.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
196,Kay Hansen,Jinh Yu Frey,-175,138,57.142857,138.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.18,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0
197,Jordan Griffin,Youssef Zalal,-106,-114,94.339623,87.719298,2020-06-27,"Las Vegas, Nevada, USA",USA,Blue,...,2.0,0.25,0.0,0.0,0.0,2.0,0.0,1.0,1.0,1
198,Curtis Blaydes,Alexander Volkov,-400,325,25.000000,325.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,2.0,0.56,0.5,0.0,0.0,3.0,0.0,0.0,0.0,0


In [11]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


In [12]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
56,Colby Covington,Tyron Woodley,-335,260,29.850746,260.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
57,Khamzat Chimaev,Gerald Meerschaert,-400,300,25.000000,300.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
58,Johnny Walker,Ryan Spann,-125,100,80.000000,100.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
59,Mackenzie Dern,Randa Markos,-177,140,56.497175,140.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
60,Kevin Holland,Darren Stewart,-265,205,37.735849,205.000000,2020-09-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Takashi Sato,Jason Witt,-278,215,35.971223,215.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
196,Kay Hansen,Jinh Yu Frey,-175,138,57.142857,138.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.18,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0
197,Jordan Griffin,Youssef Zalal,-106,-114,94.339623,87.719298,2020-06-27,"Las Vegas, Nevada, USA",USA,Blue,...,2.0,0.25,0.0,0.0,0.0,2.0,0.0,1.0,1.0,1
198,Curtis Blaydes,Alexander Volkov,-400,325,25.000000,325.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,2.0,0.56,0.5,0.0,0.0,3.0,0.0,0.0,0.0,0


In [13]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [14]:
models[1][model_num]

"RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)"

In [15]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [16]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [17]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [18]:
#1. set features
my_pos_features = ['R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [19]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_10_27_20
RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,
                       n_jobs=-1, random_state=75)
['R_Height_cms', 'R_Bantamweight_rank', 'B_current_lose_streak', 'avg_sub_att_dif', 'B_Middleweight_rank', 'R_current_win_streak', 'total_round_dif', 'B_avg_SUB_ATT', 'B_avg_SIG_STR_landed', 'age_dif', 'B_Light Heavyweight_rank', 'R_avg_TD_pct', 'R_ev', "B_Women's Flyweight_rank", "B_Women's Strawweight_rank", 'B_Bantamweight_rank', 'R_Reach_cms', 'B_win_by_Decision_Unanimous', 'constant_1']
1


In [20]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [21]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [22]:
"""keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    
    """
    

'keep_going = True\nwhile(keep_going):\n    #1. Set Features\n    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)\n    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, \n                                             min_ev=test_model_ev))\n    print_model()\n    save_model()\n    #2 Set hyperparameters\n    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):\n    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, \n                                      min_ev=test_model_ev)\n    print_model()\n    save_model()\n    #3. Set EV\n    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):\n    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)\n\n    old_test_model_features = test_model_features #Thi

In [23]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3190, 19)

(3190,)

(3190, 2)

(93, 19)

(93,)

(93, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-335,260,0.802213,0.197787,0
1,-400,300,0.750679,0.249321,0
2,-125,100,0.489488,0.510512,0
3,-177,140,0.585749,0.414251,0
4,-265,205,0.794530,0.205470,0
...,...,...,...,...,...
88,-278,215,0.729656,0.270344,0
89,-175,138,0.429682,0.570318,0
90,-106,-114,0.548308,0.451692,1
91,-400,325,0.718764,0.281236,0



          Number of matches: 93
          Number of bets: 74
          Number of winning bets: 39
          Number of losing bets: 35
          Number of underdog bets: 47
          Number of underdog wins: 21
          Number of underdog losses: 26
          Number of Favorite bets: 23
          Number of favorite wins: 17
          Number of favorite losses: 6
          Number of even bets: 4
          Number of even wins: 1
          Number of even losses: 3
          Profit: 1462.2563187161593
          Profit per bet: 19.760220523191343
          Profit per match: 15.723186222754402
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [24]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [25]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [26]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [27]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  'DecisionTreeClassifier(max_depth=5, max_leaf_nodes=5, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)',
  'GradientBoostingClassifier(max_depth=95, min_sample